# Trader Analysis

Transform raw fills into trader metrics.

In [ ]:
import json
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', None)
SAMPLES_DIR = Path('../hyperliquid_samples/node_fills_by_block')

---

## Load & Transform

In [ ]:
# Load fills from sample JSON
fills = []
for path in SAMPLES_DIR.glob('*.json'):
    with open(path) as f:
        fills.extend(json.load(f))

df = pd.DataFrame(fills)
print(f"Loaded {len(df):,} fills")

In [ ]:
# Type conversions
df['time'] = pd.to_datetime(df['time'], unit='ms')
df['px'] = pd.to_numeric(df['px'])
df['sz'] = pd.to_numeric(df['sz'])
df['closedPnl'] = pd.to_numeric(df['closedPnl'], errors='coerce').fillna(0)
df['fee'] = pd.to_numeric(df['fee'], errors='coerce').fillna(0)

# Derived columns
df['volume'] = df['px'] * df['sz']
df['is_maker'] = ~df['crossed']
df['is_close'] = df['dir'].str.startswith('Close')
df['is_win'] = (df['closedPnl'] > 0) & df['is_close']

df[['user', 'coin', 'px', 'sz', 'dir', 'closedPnl', 'fee', 'crossed']].head()

---

## Trader Profiles

Aggregate fills into per-trader metrics.

In [ ]:
# Base aggregations
profiles = df.groupby('user').agg(
    volume=('volume', 'sum'),
    trades=('volume', 'count'),
    realized_pnl=('closedPnl', 'sum'),
    fees=('fee', 'sum'),
    maker_trades=('is_maker', 'sum'),
    coins_traded=('coin', 'nunique')
)

# Win stats (closes only)
closes = df[df['is_close']]
win_stats = closes.groupby('user').agg(
    total_closes=('is_win', 'count'),
    wins=('is_win', 'sum')
)

# Combine
profiles = profiles.join(win_stats)
profiles['net_pnl'] = profiles['realized_pnl'] - profiles['fees']
profiles['maker_pct'] = profiles['maker_trades'] / profiles['trades'] * 100
profiles['win_rate'] = profiles['wins'] / profiles['total_closes'] * 100

profiles.sort_values('net_pnl', ascending=False).head(10)

---

## Quick Stats

In [ ]:
print(f"Fills: {len(df):,}")
print(f"Traders: {df['user'].nunique():,}")
print(f"Coins: {df['coin'].nunique():,}")
print(f"Volume: ${df['volume'].sum():,.0f}")
print(f"Time range: {df['time'].min()} → {df['time'].max()}")